In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv

URL = "https://nol.interpark.com/"

service = Service(ChromeDriverManager().install())
options = Options()

options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=service, options=options)
driver.get(URL)
time.sleep(2)

ticket_link = driver.find_element(By.CSS_SELECTOR, 'a[href*="tickets.interpark.com"]')
href = ticket_link.get_attribute("href")
driver.get(href)
time.sleep(2)

wait = WebDriverWait(driver, 10)

musical_tab = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, 'a[href*="genre/musical"]')
))
musical_tab.click()

wait.until(EC.url_contains("/contents/genre/musical"))

list_container = wait.until(EC.presence_of_element_located(
    (By.CSS_SELECTOR, 'div[class*="ProductList_contents__"]')
))


y = list_container.location['y']
driver.execute_script(f"window.scrollTo(0, {y});")
time.sleep(1)

def has_items_in_container(driver):
    items = list_container.find_elements(By.CSS_SELECTOR, 'a[role="link"][tabindex="0"]')
    return len(items) > 0

wait.until(has_items_in_container)

items = []
while len(items) < 20:
    items = list_container.find_elements(By.CSS_SELECTOR, 'a[role="link"][tabindex="0"]')
    if len(items) >= 20:
        break
    driver.execute_script("window.scrollBy(0, 1088);")
    time.sleep(0.8)

items = items[:20]

data = []
for a in items:
    lis = a.find_elements(By.CSS_SELECTOR, "ul li")
    if len(lis) >= 3:
        title = lis[0].text.strip()
        place = lis[1].text.strip()
        period = lis[2].text.strip()
        data.append((title, place, period))

for i, (title, place, period) in enumerate(data, start=1):
    print(f"No.{i} {title}-{place}-{period}")

csv_path = "interpark_musical_top20.csv"
with open(csv_path, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["No", "공연명", "공연장", "공연기간"])
    for i, (title, place, period) in enumerate(data, start=1):
        writer.writerow([i, title, place, period])

print(f"\n✅ CSV 저장 완료: {csv_path}")

driver.quit()

No.1 이터니티-NOL 유니플렉스 1관-2025.12.27 - 2026.03.15
No.2 제임스 바이런 딘-극장 온-2026.01.09 - 2026.03.01
No.3 전설의 리틀 농구단-플러스씨어터-2026.03.10 - 2026.05.25
No.4 너를 위한 글자-링크아트센터드림 드림4관-2026.03.04 - 2026.05.31
No.5 디아길레프-예스24아트원 1관-2026.03.31 - 2026.06.14
No.6 로미오와 줄리엣-한전아트센터-2026.03.24 - 2026.05.31
No.7 태권, 날아올라-국립중앙박물관 극장 용-2026.04.04 - 2026.05.31
No.8 로빈-대학로 TOM(티오엠) 1관-2025.12.01 - 2026.03.01
No.9 존 도우-NOL 서경스퀘어 스콘 1관-2025.12.10 - 2026.03.01
No.10 시지프스-예스24스테이지 2관-2025.12.16 - 2026.03.08
No.11 타오르는 어둠 속에서-링크아트센터 페이코홀-2025.12.10 - 2026.03.08
No.12 난쟁이들-플러스씨어터-2025.11.05 - 2026.03.01
No.13 말리-백암아트홀-2025.12.20 - 2026.02.15
No.14 캐빈-et theatre 1 (이티 씨어터 원)-2025.11.27 - 2026.03.01
No.15 은하철도의 밤-예스24스테이지 1관-2025.12.23 - 2026.03.15
No.16 헝키쇼 (HUNKY SHOW)-스타스테이지-2025.04.25 - 2026.03.31
No.17 더 맨 얼라이브 : 초이스-서울숲씨어터 1관-2025.11.05 - 2026.03.31
No.18 테이크 오프-대학로 스타시티-2025.11.14 - 2026.02.08
No.19 스폿라이트-뮤지컬펍 스폿라이트-2025.09.02 - 2026.02.28
No.20 김종욱 찾기-브릭스씨어터-OPEN RUN

✅ CSV 저장 완료: interpark_musical_top20.csv
